In [1]:
import pandas as pd
# Absolute path example
data = pd.read_csv("recipe_final (1).csv")
data.head()

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,"['pork belly', 'smoked paprika', 'kosher salt'..."
1,1,240488,"Pork Loin, Apples, and Sauerkraut",4.76,https://images.media-allrecipes.com/userphotos...,29,19,18,10,73,33,104,41,"['sauerkraut drained', 'Granny Smith apples sl..."
2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,"['chicken wings', 'sprigs rosemary', 'head gar..."
3,3,87211,Chicken Pesto Paninis,4.62,https://images.media-allrecipes.com/userphotos...,163,32,45,20,65,20,43,18,"['focaccia bread quartered', 'prepared basil p..."
4,4,245714,Potato Bacon Pizza,4.50,https://images.media-allrecipes.com/userphotos...,2,8,12,5,14,7,8,3,"['red potatoes', 'strips bacon', 'Sauce:', 'he..."


In [3]:
data.isnull().sum()

Unnamed: 0          0
recipe_id           0
recipe_name         0
aver_rate           0
image_url           0
review_nums         0
calories            0
fat                 0
carbohydrates       0
protein             0
cholesterol         0
sodium              0
fiber               0
ingredients_list    0
dtype: int64

In [25]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(data['ingredients_list'])

0        ['pork belly', 'smoked paprika', 'kosher salt'...
1        ['sauerkraut drained', 'Granny Smith apples sl...
2        ['chicken wings', 'sprigs rosemary', 'head gar...
3        ['focaccia bread quartered', 'prepared basil p...
4        ['red potatoes', 'strips bacon', 'Sauce:', 'he...
                               ...                        
48730    ['fluid ounce tequila', 'fluid ounce vodka', '...
48731    ['raisins', 'brandy', 'cream cheese', 'white s...
48732    ['sliced peaches drained', 'scoops vanilla ice...
48733    ['butter', 'habanero peppers', 'fresh peaches'...
48734    ['olive oil', 'bulb garlic', 'tomatoes chopped...
Name: ingredients_list, Length: 48735, dtype: object

In [7]:
scaler = StandardScaler()
X_numerical = scaler.fit_transform(data[['calories', 'fat', 'carbohydrates', 'protein', 'cholesterol', 'sodium', 'fiber']])


In [9]:
X_numerical

array([[-0.1317045 ,  0.46001924, -1.15482863, ...,  0.04256474,
         1.13990476, -0.76393724],
       [ 0.23857551, -0.33625589, -0.01920347, ...,  0.52871248,
         1.59202345,  2.53220175],
       [ 0.05343551,  0.46001924, -1.02864806, ...,  0.16410167,
         0.15703804, -0.59490447],
       ...,
       [-0.77969453, -0.82286847, -0.27156462, ..., -0.68665688,
        -0.39336732, -0.25683894],
       [ 0.33114552,  0.32730672,  0.73787996, ...,  0.20461398,
        -0.31473798, -0.51038809],
       [-1.33511455, -0.9998185 , -1.02864806, ..., -0.80819381,
        -0.13782197, -0.42587171]])

In [13]:
import numpy as np
X_combined = np.hstack([X_numerical, X_ingredients.toarray()])

# Train KNN Model
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean', n_neighbors=3)

In [17]:
def recommend_recipes(input_features):
    input_features_scaled = scaler.transform([input_features[:7]])
    input_ingredients_transformed = vectorizer.transform([input_features[7]])
    input_combined = np.hstack([input_features_scaled, input_ingredients_transformed.toarray()])
    distances, indices = knn.kneighbors(input_combined)
    recommendations = data.iloc[indices[0]]
    return recommendations[['recipe_name', 'ingredients_list', 'image_url']]

In [19]:
input_features=[15,36,1,42,21,81,2,'pork belly, smoked paprika, kosher salt']

In [21]:
recommendations=recommend_recipes(input_features)
recommendations

C:\Users\Darsh\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,recipe_name,ingredients_list,image_url
0,Homemade Bacon,"['pork belly', 'smoked paprika', 'kosher salt'...",https://images.media-allrecipes.com/userphotos...
3507,Char Siu (Chinese Barbeque Pork),"['honey', 'Shaoxing cooking wine', 'hoisin sau...",https://images.media-allrecipes.com/userphotos...
1926,Adobo Twist,"['vegetable oil', 'onion', 'head garlic', 'por...",https://images.media-allrecipes.com/userphotos...
